In [94]:
# Coronavirus is a hoax - Politically Incorrect - 4chan
# https://boards.4chan.org/pol/thread/252406368/coronavirus-is-a-hoax#p252406757
# &#039;, 
from bs4 import BeautifulSoup
from newspaper import Article
import requests
import pandas as pd
import re

headers = requests.utils.default_headers()
headers.update({ 'User-Agent': 'Mozilla/5.0 (X11; Ubuntu; Linux x86_64; rv:52.0) Gecko/20100101 Firefox/52.0'})
url = "https://boards.4chan.org/pol//thread/252821554/new-york-state-now-has-more-coronavirus-cases"
req = requests.get(url, headers)
soup = BeautifulSoup(req.content, 'html.parser')

list_of_posts = []
gg = soup.find_all("div")
for g in gg:
    y = g.get("class")
    if y == ['post','reply']:
        list_of_posts.append(g)

chan4 = {"date_utc":[], "user":[], "responding_to":[], "text":[]}

for post in list_of_posts:
    date = None
    try:
        id_ = post.find("input").get("name")
        message = post.find('blockquote', attrs={'class':'postMessage'})
        result = re.search('<br/>(.*)</blockquote>', str(message))
        result_ = result.group(1)
        result_ = re.sub("<.*?>","",result_)
        result_ = re.sub("&#039;","",result_)
        result_ = re.sub("&gt;","",result_)
        date_utc = post.find('span', attrs={'class':'dateTime'}).get("data-utc")
        hyper_linked_user = post.find('a', attrs={'class':'quotelink'}).get("href")
        
        chan4["date_utc"].append(date_utc)
        chan4["user"].append(id_)
        chan4["responding_to"].append(hyper_linked_user[2:])
        chan4["text"].append(result_)

    except:
        pass

Data = pd.DataFrame.from_dict(chan4)
# chan4_new_york_has_more_corona.csv
# Data.to_csv("chan4_new_york_has_more_corona.csv")




In [101]:
import pandas as pd
data = pd.read_csv('Data/Raw_Data/Chan4/chan4_new_york_has_more_corona.csv')  

import re
alphabets= "([A-Za-z])"
prefixes = "(Mr|St|Mrs|Ms|Dr)[.]"
suffixes = "(Inc|Ltd|Jr|Sr|Co)"
starters = "(Mr|Mrs|Ms|Dr|He\s|She\s|It\s|They\s|Their\s|Our\s|We\s|But\s|However\s|That\s|This\s|Wherever)"
acronyms = "([A-Z][.][A-Z][.](?:[A-Z][.])?)"
websites = "[.](com|net|org|io|gov)"
digits = "([0-9])"

def split_into_sentences(text):
    text = " " + text + "  "
    text = text.replace("\n"," ")
    text = re.sub(prefixes,"\\1<prd>",text)
    text = re.sub(websites,"<prd>\\1",text)
    if "Ph.D" in text: text = text.replace("Ph.D.","Ph<prd>D<prd>")
        
    text = re.sub("\s" + alphabets + "[.] "," \\1<prd> ",text)
    text = re.sub(acronyms+" "+starters,"\\1<stop> \\2",text)
    text = re.sub(alphabets + "[.]" + alphabets + "[.]" + alphabets + "[.]","\\1<prd>\\2<prd>\\3<prd>",text)
    text = re.sub(alphabets + "[.]" + alphabets + "[.]","\\1<prd>\\2<prd>",text)
    text = re.sub(" "+suffixes+"[.] "+starters," \\1<stop> \\2",text)
    text = re.sub(" "+suffixes+"[.]"," \\1<prd>",text)
    text = re.sub(" " + alphabets + "[.]"," \\1<prd>",text)
    text = re.sub(digits + "[.]" + digits,"\\1<prd>\\2",text)
    
    if "”" in text: text = text.replace(".”","”.")
    if "\"" in text: text = text.replace(".\"","\".")
    if "!" in text: text = text.replace("!\"","\"!")
    if "?" in text: text = text.replace("?\"","\"?")
        
    text = text.replace(".",".<stop>")
    text = text.replace("?","?<stop>")
    text = text.replace("!","!<stop>")
    text = text.replace("<prd>",".")
    
    sentences = text.split("<stop>")
    sentences = sentences[:-1]
    sentences = [s.strip() for s in sentences]
    
    return sentences

In [102]:
with open("Data/Extracted_Data/chan4/into_relex.txt", "a+") as file_object:
    file_object.write("text\n")
    for _,row in data.iterrows():
        file_object.write(str(row["text"]) + "\n") 